In [3]:
import os
import shutil
from multiprocessing import Pool

import librosa
import pandas as pd
import soundfile as sf

from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

language = 'Tamil'
lang = 'ta'

In [ ]:
# # only for Odia and Marathi

# def clean_id(text, prefix):
#     if 'text' in text:
#         text = text.strip()
#         return text.replace('text', f"{prefix}_")
#     return text

# data_dir = f"/nlsasfs/home/ai4bharat/manidl/ttsteam/datasets/Indic TTS Data/TTS_data_Phase2_to_be_copied/{language}" 
# for speaker in ['male', 'female']:
#     data_dir_speaker = f"{data_dir}/{speaker}/mono"
#     df = pd.read_csv(f'{data_dir_speaker}/txt.done.data', sep='"', usecols=[0,1], header=None, names=['id', 'text'])
#     df['id'] = df['id'].apply(lambda text: clean_id(text, f"train_{language.lower()}{speaker}"))
#     df.to_csv(f'{data_dir_speaker}/txt.done.data', sep='"', header=None, index=None)

#     wav_dir = f"{data_dir_speaker}/wav"
#     for fn in os.listdir(wav_dir):
#         fn_new = clean_id(fn, f"train_{language.lower()}{speaker}")
#         fp_src = os.path.join(wav_dir, fn)
#         fp_dst = os.path.join(wav_dir, fn_new)
#         os.replace(fp_src, fp_dst)

# IndicTTS in LJSpeech format

In [4]:
data_dir = f"/home/speech/ttsteam/datasets/indictts/{lang}" # update the path
data_dir_new = f"/home/speech/ttsteam/datasets/indictts/{lang}" # update the path

In [3]:
#os.makedirs(data_dir_new)

In [4]:
shutil.copytree(f'{data_dir}/male/mono/wav', f'{data_dir_new}/wavs')

'/home/speech/ttsteam/datasets/indictts/ta/wavs'

In [5]:
shutil.copytree(f'{data_dir}/female/mono/wav', f'{data_dir_new}/wavs', dirs_exist_ok=True)

'/home/speech/ttsteam/datasets/indictts/ta/wavs'

In [6]:
metadata_male_fp = f"{data_dir}/male/mono/txt.done.data"
metadata_male = pd.read_csv(metadata_male_fp, sep='"', usecols=[0,1], header=None)
metadata_male[0] = metadata_male[0].str.replace('\(', '').str.strip()
metadata_male[1] = metadata_male[1].str.strip()
metadata_male[2] = 'male'
print(metadata_male.shape)
metadata_male.head()

(3717, 3)


,0,1,2
0,train_tamilmale_00001,அது தஞ்சாவூர்க் கோட்டைக்குள் பிரவேசிக்கவும் சக...,male
1,train_tamilmale_00002,அதற்குத் தகுந்தபடி ஏதாவது கொஞ்சம் பேசி வேஷம் ப...,male
2,train_tamilmale_00003,ஆனால் அவன் எதிர்பார்த்த சந்தர்ப்பம் ஒன்றும் கி...,male
3,train_tamilmale_00004,அப்படியும் பல்லக்கு கீழே வைக்கப்படவில்லை ஒரே ம...,male
4,train_tamilmale_00005,கோட்டைக்குள் பல்லக்குப் போய்விட்டால் அப்புறம் ...,male


In [7]:
metadata_female_fp = f"{data_dir}/female/mono/txt.done.data"
metadata_female = pd.read_csv(metadata_female_fp, sep='"', usecols=[0,1], header=None)
metadata_female[0] = metadata_female[0].str.replace('\(', '').str.strip()
metadata_female[1] = metadata_female[1].str.strip()
metadata_female[2] = 'female'
print(metadata_female.shape)
metadata_female.head()

(3243, 3)


,0,1,2
0,train_tamilfemale_00001,"அதற்குத் தகுந்தபடி, ஏதாவது கொஞ்சம் பேசி, வேஷம்...",female
1,train_tamilfemale_00002,"ஆனால், அவன் எதிர்பார்த்த சந்தர்ப்பம் ஒன்றும், ...",female
2,train_tamilfemale_00003,"அப்படியும், பல்லக்கு கீழே வைக்கப்படவில்லை ஒரே ...",female
3,train_tamilfemale_00004,"கோட்டைக்குள் பல்லக்குப் போய்விட்டால், அப்புறம்...",female
4,train_tamilfemale_00005,"எடுத்த காரியத்தை முடிக்காமல், உயிரோடு திரும்பி...",female


In [8]:
metadata = pd.concat([metadata_male, metadata_female]).reset_index(drop=True)
metadata

,0,1,2
0,train_tamilmale_00001,அது தஞ்சாவூர்க் கோட்டைக்குள் பிரவேசிக்கவும் சக...,male
1,train_tamilmale_00002,அதற்குத் தகுந்தபடி ஏதாவது கொஞ்சம் பேசி வேஷம் ப...,male
2,train_tamilmale_00003,ஆனால் அவன் எதிர்பார்த்த சந்தர்ப்பம் ஒன்றும் கி...,male
3,train_tamilmale_00004,அப்படியும் பல்லக்கு கீழே வைக்கப்படவில்லை ஒரே ம...,male
4,train_tamilmale_00005,கோட்டைக்குள் பல்லக்குப் போய்விட்டால் அப்புறம் ...,male
...,...,...,...
6955,train_tamilfemale_03239,"பின்பு அவர், துரியோதனனுடன் சதுரங்கம் ஆடி, அதில...",female
6956,train_tamilfemale_03240,"பெட்டிக்குள்ளிருந்த, கல்லில்தான் ஏதோ மாயசக்தி ...",female
6957,train_tamilfemale_03241,"அது போல, சத்துருஜித்தின் தாத்தா யுதாஜித், உஜ்ஜ...",female
6958,train_tamilfemale_03242,"அது கேட்டு கும்பகர்ணன், நீங்கள் சீதையைக் கவர்ந...",female


In [9]:
metadata.to_csv(f'{data_dir_new}/metadata.csv', sep='|', index=False, header=False)

## Resampling

In [10]:
os.makedirs(f'{data_dir_new}/wavs-20k')

In [5]:
def resample_file(func_args):
    fp_src, fp_dst, output_sr = func_args
    y, sr = librosa.load(fp_src, sr=output_sr)
    sf.write(fp_dst, y, sr)

In [6]:
fps_src = [f'{data_dir_new}/wavs/{fn}' for fn in tqdm(os.listdir(f'{data_dir_new}/wavs'))]
fps_dst = [f'{data_dir_new}/wavs-20k/{fn}' for fn in tqdm(os.listdir(f'{data_dir_new}/wavs'))]
srs = [22050] * len(fps_src)

  0%|          | 0/6960 [00:00<?, ?it/s]

  0%|          | 0/6960 [00:00<?, ?it/s]

In [7]:
# audio_files = list(zip(fps_src, fps_dst, srs))
# with Pool(processes=16) as p:
#     with tqdm(total=len(fps_src)) as pbar:
#         for i, _ in enumerate(p.imap_unordered(resample_file, audio_files)):
#             pbar.update()

In [8]:
audio_files = list(zip(fps_src, fps_dst, srs))
for audio_file in tqdm(audio_files):
    resample_file(audio_file)

  0%|          | 0/6960 [00:00<?, ?it/s]

## Format data for eval

In [10]:
data_dir = f"/home/speech/ttsteam/datasets/indictts/{lang}"

In [11]:
df = pd.read_csv(f'{data_dir}/metadata.csv', sep='|', names=['id', 'text', 'speaker'])
print(df.shape)
df.head()

(6960, 3)


,id,text,speaker
0,train_tamilmale_00001,அது தஞ்சாவூர்க் கோட்டைக்குள் பிரவேசிக்கவும் சக...,male
1,train_tamilmale_00002,அதற்குத் தகுந்தபடி ஏதாவது கொஞ்சம் பேசி வேஷம் ப...,male
2,train_tamilmale_00003,ஆனால் அவன் எதிர்பார்த்த சந்தர்ப்பம் ஒன்றும் கி...,male
3,train_tamilmale_00004,அப்படியும் பல்லக்கு கீழே வைக்கப்படவில்லை ஒரே ம...,male
4,train_tamilmale_00005,கோட்டைக்குள் பல்லக்குப் போய்விட்டால் அப்புறம் ...,male


In [12]:
df_train, df_test = train_test_split(df, test_size=0.01, stratify=df['speaker'], random_state=0)
print(len(df_train), len(df_test))

6890 70


In [13]:
df_train.to_csv(f'{data_dir}/metadata_train.csv', sep='|', index=False, header=False)
df_test.to_csv(f'{data_dir}/metadata_test.csv', sep='|', index=False, header=False)

In [14]:
df_train[df_train['speaker']=='male'].to_csv(f'{data_dir}/metadata_train_male.csv', sep='|', index=False, header=False)
df_test[df_test['speaker']=='male'].to_csv(f'{data_dir}/metadata_test_male.csv', sep='|', index=False, header=False)

df_train[df_train['speaker']=='female'].to_csv(f'{data_dir}/metadata_train_female.csv', sep='|', index=False, header=False)
df_test[df_test['speaker']=='female'].to_csv(f'{data_dir}/metadata_test_female.csv', sep='|', index=False, header=False)

In [ ]:
os.makedirs(f'{data_dir}/wavs-20k-test-male/')
os.makedirs(f'{data_dir}/wavs-20k-test-female/')

In [16]:
for row_id, row in tqdm(df_test.iterrows(), total=len(df_test)):
    src = f'{data_dir}/wavs-20k/{row["id"]}.wav'
    dst = f'{data_dir}/wavs-20k-test-{row["speaker"]}/{row["id"]}.wav'
    shutil.copyfile(src, dst)

  0%|          | 0/70 [00:00<?, ?it/s]